# Pipeline overview:
1. **Find a query protein in the orthoDB database** (retrieve the corresponding orthoDB ID)
   - you can find a protein by looking up its uniprot ID in the orthoDB database. You can also set the orthoDB ID manually if you already know it. 
   - *Note: If it can't find an orthoDB ID for a given UniprotID, it doesn't mean that the protein is absent from the orthoDB. It could still be present but was retrieved from a different database and a uniprot ID was not mapped to it. View OrthoDB documentation for more info on where the sequences come from. I have not solved this problem. In the future, it would be nice to develop a way to search for the actual full length sequence using blast or something, if it fails to find the uniprot ID*
2. **Retrieve the orthoDB-defined groups of homologous sequences** (orthogroup IDs) containing the query protein
3. **Select a group based on phylogenetic level**
4. **Filter** out sequences in the group that are too short (relative to the length of the query sequence) or that contain non amino acid characters
5. **Filter to least divergent orthologs (LDOs)**:
   - For each organism in the group, select the sequence that is most similar to the query sequence such that there is only one sequence per organism
6. **Cluster the filtered LDOs using CD-HIT**
7. **Align the clustered sequences** (uses MAFFT by default)
8. **output** the alignment and the ortholog group information in a directory structure that is compatible with the conservation analysis pipeline (link)
    - the group information is output in the form of a json file that can be imported as a python object (see `./examples/` for examples of how to use the object)


---

Here we will go through each step and explain what the pipeline does. <br>
You don't have to worry about using any of this code directly, since you should be able to just run the script but it might be useful to understand what is going on under the hood if you want to write your own script.<br>
There main script in `../orthoDB_construct_groups_for_all_human_genes/` combines all of the steps in a single script, using a configuration file to specify all of the parameters.<br>
This notebook will explain what each of the parameters does?

# imports

In [3]:
import pandas as pd
import local_env_variables.env_variables as env
import local_orthoDB_group_tools.sql_queries as sql_queries
import local_orthoDB_group_tools.uniprotid_search as uniprotid_search

%load_ext autoreload
%autoreload 2

# 1. find a query protein in the orthoDB database

## search for the uniprot id: Q8TC90

In [4]:
uniprot_id = 'Q8TC90'
print(uniprotid_search.uniprotid_2_odb_gene_id(uniprot_id))
odb_gene_id = uniprotid_search.uniprotid_2_odb_gene_id(uniprot_id)

9606_0:002f40


## get the ortholog groups (og_ids) that contain the query protein

In [5]:
og_ids = sql_queries.odb_gene_id_2_ogid_list(odb_gene_id)
print(og_ids)

['96736at314146', '351562at32523', '4642869at2759', '869863at7742', '18706at314295', '96736at9443', '1771281at33208', '622381at9347', '18706at9604', '21856at40674']


In [6]:
# og_id information
sql_queries.ogid_list_2_og_info_df(og_ids)

,OG id,level NCBI tax id,level name,total non-redundant count of species underneath,OG name
8,18706at9604,9604,Hominidae,5.0,Coiled-coil domain-containing glutamate-rich p...
4,18706at314295,314295,Hominoidea,7.0,Coiled-coil domain-containing glutamate-rich p...
5,96736at9443,9443,Primates,30.0,coiled-coil glutamate rich protein 1
0,96736at314146,314146,Euarchontoglires,70.0,coiled-coil glutamate rich protein 1
7,622381at9347,9347,Eutheria,182.0,coiled-coil glutamate rich protein 1
9,21856at40674,40674,Mammalia,191.0,coiled-coil glutamate rich protein 1
1,351562at32523,32523,Tetrapoda,325.0,coiled-coil glutamate rich protein 1
3,869863at7742,7742,Vertebrata,470.0,coiled-coil glutamate rich protein 1
6,1771281at33208,33208,Metazoa,817.0,coiled-coil glutamate rich protein 1
2,4642869at2759,2759,Eukaryota,1952.0,coiled-coil glutamate rich protein 1
